In [1]:
import pandas as pd
import re
import json
import csv
from datetime import datetime
import logging

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
orders_json = [
    {
        "Order": "92022795",
        "Delivered": "The order was delivered at 11:45 PM on October 2, 2023.",
        "Pick Up Location": "800 W Interstate 20, Big Spring, TX 79720, USA",
        "Order Details": {
            "1 \u00d7 Barcel Takis Fuego Hot Chili Pepper & Lime Corn Snacks (4 oz) (Chips & Popcorn TX)": "$4.19",
            " 1 \u00d7 Hostess Powdered Donettes Bag (10.5oz) (Donuts TX)": "$6.39",
            " 1 \u00d7 Big Red Bottle (20oz) (Soda TX)": "$3.79",
            " 1 \u00d7 Cheetos Crunchy Jalapeno (8.5 oz) (Chips & Popcorn TX)": "$3.19"
        },
        "Subtotal": "$17.56",
        "Subtotal Tax": "$0.31",
        "Commission (23%)": "-$4.04",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$13.83",
        "Transaction #8120214500 - Delivery": "$13.83"
    },
    {
        "Order": "3286DC34",
        "Delivered": "The order was delivered at 11:45 PM on October 2, 2023.",
        "Pick Up Location": "145 Lehman Rd, Kyle, TX 78640, USA",
        "Order Details": {
            "1 \u00d7 Ben & Jerry's Chunky Monkey Ice Cream (16 oz) (Ice Cream TX)": "$12.49"
        },
        "Subtotal": "$12.49",
        "Subtotal Tax": "$0.00",
        "Commission (23%)": "-$2.87",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$9.62",
        "Transaction #8120252453 - Delivery": "$9.62"
    },
    {
        "Order": "F9A3C1CE",
        "Delivered": "The order was delivered at 11:36 PM on October 2, 2023.",
        "Pick Up Location": "800 W Interstate 20, Big Spring, TX 79720, USA",
        "Order Details": {
            "1 \u00d7 Coors Original Beer Bottle (12oz x 6ct) (Domestic Beer 70)": "$13.79"
        },
        "Subtotal": "$13.79",
        "Subtotal Tax": "$1.14",
        "Commission (16%)": "-$2.21",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$12.72",
        "Transaction #8120214907 - Delivery": "$12.72"
    },
    {
        "Order": "8BE8532F",
        "Delivered": "The order was delivered at 11:14 PM on October 2, 2023.",
        "Pick Up Location": "14620 Ranch Rd 12, Wimberley, TX 78676, USA",
        "Order Details": {
            "1 \u00d7 Chicken Tender Meal Kit (Meal Kits TXB#57)": "$9.19",
            " 3 \u00d7 Cayman Jack Margarita Can (19 oz) (Flavored Malt Beverages CT)": "$17.07"
        },
        "Subtotal": "$26.26",
        "Subtotal Tax": "$2.17",
        "Commission (23%)": "-$6.04",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$22.39",
        "Transaction #8119974498 - Delivery": "$22.39"
    },
    {
        "Order": "D7BD216C",
        "Delivered": "The order was delivered at 11:05 PM on October 2, 2023.",
        "Pick Up Location": "5004 Wesley St, Greenville, TX 75402, USA",
        "Order Details": {
            "1 \u00d7 Michelob Ultra Light Beer Can (12 oz x 6 ct) (Domestic Beer NT)": "$16.29"
        },
        "Subtotal": "$16.29",
        "Subtotal Tax": "$1.34",
        "Commission (23%)": "-$3.75",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$13.88",
        "Transaction #8120073615 - Delivery": "$13.88"
    },
    {
        "Order": "8C4F6250",
        "Delivered": "The order was delivered at 5:30 PM on October 2, 2023.",
        "Pick Up Location": "816 N Center St, Bonham, TX 75418, USA",
        "Order Details": {
            "1 \u00d7 Coors Light Beer Can (12 oz x 18 ct) (Domestic Beer NT)": "$28.79"
        },
        "Subtotal": "$28.79",
        "Subtotal Tax": "$2.38",
        "Commission (16%)": "-$4.61",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$26.56",
        "Transaction #8117259781 - Delivery": "$26.56"
    },
    {
        "Order": "0ED5F1C8",
        "Delivered": "The order was delivered at 4:12 PM on October 2, 2023.",
        "Pick Up Location": "816 N Center St, Bonham, TX 75418, USA",
        "Order Details": {
            "1 \u00d7 Coors Light Beer Can (12 oz x 18 ct) (Domestic Beer NT)": "$28.79"
        },
        "Subtotal": "$28.79",
        "Subtotal Tax": "$2.38",
        "Commission (16%)": "-$4.61",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$26.56",
        "Transaction #8116747084 - Delivery": "$26.56"
    },
    {
        "Order": "60BF65AB",
        "Delivered": "The order was delivered at 2:45 PM on October 2, 2023.",
        "Pick Up Location": "13006 Bandera Rd, Helotes, TX 78023, USA",
        "Order Details": {
            "2 \u00d7 Twisted Tea Hard Iced Tea Cocktail Original (24 oz) (Flavored Malt Beverages 60)": "$11.38",
            " 2 \u00d7 Ice Small (Fountain, Frozen, & Tea)": "$9.98"
        },
        "Subtotal": "$21.36",
        "Subtotal Tax": "$1.71",
        "Commission (16%)": "-$3.42",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$19.65",
        "Transaction #8116125108 - Delivery": "$19.65"
    },
    {
        "Order": "C032FE0A",
        "Delivered": "The order was delivered at 9:24 AM on October 2, 2023.",
        "Pick Up Location": "1301 N U.S. Hwy 281, Marble Falls, TX 78654, USA",
        "Order Details": {
            "1 \u00d7 Potato Egg Cheese Taco (Breakfast Tacos & Quesadillas)": "$4.69",
            " 1 \u00d7 Sausage Egg Cheese Taco (Breakfast Tacos & Quesadillas)": "$3.79",
            " 1 \u00d7 C4 Frozen Bombscicle Can (16oz) (Energy Drinks TX)": "$4.79",
            " 1 \u00d7 C4 Skittles Can (16oz) (Energy Drinks TX)": "$4.79",
            " 1 \u00d7 C4 Starburst Cherry Can (16oz) (Energy Drinks TX)": "$4.79",
            " 1 \u00d7 Bacon Egg Cheese Taco (Breakfast Tacos & Quesadillas)": "$4.69",
            " 1 \u00d7 Breakfast Quesadilla (Breakfast Tacos & Quesadillas)": "$7.49"
        },
        "Subtotal": "$35.03",
        "Subtotal Tax": "$2.89",
        "Commission (25%)": "-$8.76",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$29.16",
        "Transaction #8114283446 - Delivery": "$29.16"
    },
    {
        "Order": "7DE4AD15",
        "Delivered": "The order was delivered at 9:21 AM on October 2, 2023.",
        "Pick Up Location": "1741 TX-121, Bonham, TX 75418, USA",
        "Order Details": {
            "1 \u00d7 Chex Mix Cheddar (3.75 oz) (Crackers & Pretzels TX)": "$4.39",
            " 1 \u00d7 Chicken Tender Meal Kit (Meal Kits TXB#10)": "$9.19",
            " Subtotal for Tax Calculations": "$13.58"
        },
        "Subtotal Tax": "$0.76",
        "Commission (25%)": "-$3.40",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$10.94",
        "Transaction #8114267746 - Delivery": "$10.94"
    },
    {
        "Order": "AF6DBE37",
        "Delivered": "The order was delivered at 7:46 AM on October 2, 2023.",
        "Pick Up Location": "2320 W Main St, Durant, OK 74701, USA",
        "Order Details": {
            "1 \u00d7 Coke Classic Bottle (20oz) (Soda OK)": "$3.19",
            " 1 \u00d7 TXB Coffee or Cappuccino (20 oz) (Brewed Coffee) Flavor: TXB Southern Pecan Coffee": "$0.00",
            " 2 \u00d7 Red Bull Energy Drink Can (12 oz) (Energy Drinks OK)": "$11.58"
        },
        "Subtotal": "$14.77",
        "Subtotal Tax": "$1.38",
        "Commission (23%)": "-$3.40",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$11.37",
        "Transaction #8114014205 - Delivery": "$11.37"
    },
    {
        "Order": "D84A9C02",
        "Cancelled - Paid": "The order was cancelled on October 2, 2023 at 6:13 AM. You\u2019re paid for cancelled orders when you\u2019ve successfully confirmed an order, it was prepared, and you did not initiate or are not at fault for the cancellation.",
        "Pick Up Location": "14620 Ranch Rd 12, Wimberley, TX 78676, USA",
        "Order Details": {
            "4 \u00d7 Cayman Jack Margarita Can (19 oz) (Flavored Malt Beverages CT)": "$22.76"
        },
        "Subtotal": "$22.76",
        "Subtotal Tax": "$1.88",
        "Commission (23%)": "-$5.23",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$19.41",
        "Transaction #8113899308 - Delivery": "$19.41"
    },
    {
        "Order": "91EC3C1E",
        "Delivered": "The order was delivered at 2:45 AM on October 2, 2023.",
        "Pick Up Location": "801 Lafayette St, Laredo, TX 78041, USA",
        "Order Details": {
            "1 \u00d7 M&M's Peanut King Size (3.27 oz) (Chocolate TX)": "$3.79",
            " 1 \u00d7 Cheetos Crunchy Flamin' Hot Limon (8.5 oz) (Chips & Popcorn TX)": "$6.49",
            " 1 \u00d7 Cheetos Crunchy Jalapeno (8.5 oz) (Chips & Popcorn TX)": "$3.19"
        },
        "Subtotal": "$13.47",
        "Subtotal Tax": "$0.31",
        "Commission (16%)": "-$2.16",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$11.62",
        "Transaction #8113712869 - Delivery": "$11.62"
    },
    {
        "Order": "4761F061",
        "Cancelled - Not Paid": "The order was cancelled on October 2, 2023 at 12:51 AM. You were not paid because the order was not able to be placed.",
        "Pick Up Location": "2196 El Indio Hwy, Eagle Pass, TX 78852, USA",
        "Order Details": {
            "1 \u00d7 Cheetos Crunchy Flamin' Hot Limon (8.5 oz) (Chips & Popcorn TX)": "$6.49",
            " 1 \u00d7 Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: Gatorade Lemon-Lime": "$0.00"
        },
        "Subtotal": "$6.49",
        "Subtotal Tax": "$0.00",
        "Commission (20%)": "-$1.30",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$0.00"
    },
    {
        "Order": "234FE808",
        "Delivered": "The order was delivered at 12:26 AM on October 2, 2023.",
        "Pick Up Location": "801 Lafayette St, Laredo, TX 78041, USA",
        "Order Details": {
            "1 \u00d7 Pepsi Can (12pk 12oz) (Soda TX)": "$11.29",
            " 1 \u00d7 Fountain Drink (Fountain, Frozen, & Tea) Drink Selection: Coke": "$0.00",
            " 1 \u00d7 Ice Small (Fountain, Frozen, & Tea)": "$4.99"
        },
        "Subtotal": "$16.28",
        "Subtotal Tax": "$1.34",
        "Commission (16%)": "-$2.60",
        "Total Customer Refund": "-$0.00",
        "Estimated Payout": "$15.02",
        "Transaction #8113483337 - Delivery": "$15.02"
    }
]
def list_of_dicts_to_json(list_of_dicts):
    try:
        json_str = json.dumps(list_of_dicts)
        return json_str
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
orders_json_serialized = list_of_dicts_to_json(orders_json)
orders_json_serialized

'[{"Order": "92022795", "Delivered": "The order was delivered at 11:45 PM on October 2, 2023.", "Pick Up Location": "800 W Interstate 20, Big Spring, TX 79720, USA", "Order Details": {"1 \\u00d7 Barcel Takis Fuego Hot Chili Pepper & Lime Corn Snacks (4 oz) (Chips & Popcorn TX)": "$4.19", " 1 \\u00d7 Hostess Powdered Donettes Bag (10.5oz) (Donuts TX)": "$6.39", " 1 \\u00d7 Big Red Bottle (20oz) (Soda TX)": "$3.79", " 1 \\u00d7 Cheetos Crunchy Jalapeno (8.5 oz) (Chips & Popcorn TX)": "$3.19"}, "Subtotal": "$17.56", "Subtotal Tax": "$0.31", "Commission (23%)": "-$4.04", "Total Customer Refund": "-$0.00", "Estimated Payout": "$13.83", "Transaction #8120214500 - Delivery": "$13.83"}, {"Order": "3286DC34", "Delivered": "The order was delivered at 11:45 PM on October 2, 2023.", "Pick Up Location": "145 Lehman Rd, Kyle, TX 78640, USA", "Order Details": {"1 \\u00d7 Ben & Jerry\'s Chunky Monkey Ice Cream (16 oz) (Ice Cream TX)": "$12.49"}, "Subtotal": "$12.49", "Subtotal Tax": "$0.00", "Commissi

In [12]:
master_dataset_file_path = '/Users/ekim/workspace/personal/dd-bot/dev/store_list.xlsx'


In [17]:
def read_masterdataset_excel():
    master_df = pd.read_excel(master_dataset_file_path, index_col=0)
    return master_df

master_df = read_masterdataset_excel()
# master_df


def add_cols_to_masterset_df(master_df):
    # Create pattern for street_num col
    street_num_pattern = r'(?P<street_num>\b\d{3,5}\b)'
    street_num_df = master_df['Address'].str.extract(street_num_pattern)

    # Add street_num col to df
    master_df = pd.concat([master_df, street_num_df], axis=1)

    # Cast int64 default dtype cols to strings for proper JOINs
    master_df['Site #'] = master_df['Site #'].astype(str)
    master_df['Zip'] = master_df['Zip'].astype(str)
    master_df['street_num'] = master_df['street_num'].astype(str)
    return master_df

master_df = add_cols_to_masterset_df(master_df)
master_df

,Site #,Site Description,Address,City,County,State,Zip,Notes,street_num
Store #,,,,,,,,,
1,1,Kwik Chek #1,2226 North Center,Bonham,Fannin,TX,75418,NaN,2226
3,3,Kwik Chek #3,971 FM 273,Bonham,Fannin,TX,75418,NaN,971
5,5,Kwik Chek #5,NaN,Bonham,Fannin,TX,75418,Not Open Yet,nan
6,6,Kwik Chek #6,816 North Center,Bonham,Fannin,TX,75418,NaN,816
7,7,Kwik Chek #7,101 Hwy 69,Celeste,Hunt,TX,75423,NaN,101
8,8,Kwik Chek #8,308 Hwy 69,Leonard,Fannin,TX,75452,NaN,308
10,10,Kwik Chek #10,1741 North Hwy 121,Bonham,Fannin,TX,75418,NaN,1741
12,12,Kwik Chek #12,102 S Hwy 78,Achille,Bryan,OK,74720,NaN,102
13,13,Kwik Chek #13,102 West Main,Calera,Bryan,OK,74730,NaN,102


In [20]:
# todo: logic to clean up data for instances of "        "Order": "No Delivery has been found."

def remove_incomplete_orders(orders):
    complete_orders = []
    for order in orders:
        print(order)
        if len(order) >= 7:
            complete_orders.append(order)
    return complete_orders

orders = remove_incomplete_orders(orders_json)
orders


{'Order': '92022795', 'Delivered': 'The order was delivered at 11:45 PM on October 2, 2023.', 'Pick Up Location': '800 W Interstate 20, Big Spring, TX 79720, USA', 'Order Details': {'1 × Barcel Takis Fuego Hot Chili Pepper & Lime Corn Snacks (4 oz) (Chips & Popcorn TX)': '$4.19', ' 1 × Hostess Powdered Donettes Bag (10.5oz) (Donuts TX)': '$6.39', ' 1 × Big Red Bottle (20oz) (Soda TX)': '$3.79', ' 1 × Cheetos Crunchy Jalapeno (8.5 oz) (Chips & Popcorn TX)': '$3.19'}, 'Subtotal': '$17.56', 'Subtotal Tax': '$0.31', 'Commission (23%)': '-$4.04', 'Total Customer Refund': '-$0.00', 'Estimated Payout': '$13.83', 'Transaction #8120214500 - Delivery': '$13.83'}
{'Order': '3286DC34', 'Delivered': 'The order was delivered at 11:45 PM on October 2, 2023.', 'Pick Up Location': '145 Lehman Rd, Kyle, TX 78640, USA', 'Order Details': {"1 × Ben & Jerry's Chunky Monkey Ice Cream (16 oz) (Ice Cream TX)": '$12.49'}, 'Subtotal': '$12.49', 'Subtotal Tax': '$0.00', 'Commission (23%)': '-$2.87', 'Total Custom

[{'Order': '92022795',
  'Delivered': 'The order was delivered at 11:45 PM on October 2, 2023.',
  'Pick Up Location': '800 W Interstate 20, Big Spring, TX 79720, USA',
  'Order Details': {'1 × Barcel Takis Fuego Hot Chili Pepper & Lime Corn Snacks (4 oz) (Chips & Popcorn TX)': '$4.19',
   ' 1 × Hostess Powdered Donettes Bag (10.5oz) (Donuts TX)': '$6.39',
   ' 1 × Big Red Bottle (20oz) (Soda TX)': '$3.79',
   ' 1 × Cheetos Crunchy Jalapeno (8.5 oz) (Chips & Popcorn TX)': '$3.19'},
  'Subtotal': '$17.56',
  'Subtotal Tax': '$0.31',
  'Commission (23%)': '-$4.04',
  'Total Customer Refund': '-$0.00',
  'Estimated Payout': '$13.83',
  'Transaction #8120214500 - Delivery': '$13.83'},
 {'Order': '3286DC34',
  'Delivered': 'The order was delivered at 11:45 PM on October 2, 2023.',
  'Pick Up Location': '145 Lehman Rd, Kyle, TX 78640, USA',
  'Order Details': {"1 × Ben & Jerry's Chunky Monkey Ice Cream (16 oz) (Ice Cream TX)": '$12.49'},
  'Subtotal': '$12.49',
  'Subtotal Tax': '$0.00',
  '

In [15]:
def order_id_to_pickup_location(orders):
    order_id_to_pickup_location  = {}
    for order in orders:
        print(type(order))
        order_id = order['Order']
        store_addrs = order['Pick Up Location']
        order_id_to_pickup_location[order_id] = store_addrs
    return order_id_to_pickup_location
order_to_location = order_id_to_pickup_location(orders)
order_to_location

<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>
<class 'dict'>


{'42C2A525': '5004 Wesley St, Greenville, TX 75402, USA',
 'AA85B890': '5004 Wesley St, Greenville, TX 75402, USA',
 '773C7330': '5004 Wesley St, Greenville, TX 75402, USA',
 'EDCB46B1': '1001 W Van Alstyne Pkwy, Van Alstyne, TX 75495, USA',
 '68944F97': '800 W Interstate 20, Big Spring, TX 79720, USA',
 'ED39CFE2': '1301 N U.S. Hwy 281, Marble Falls, TX 78654, USA',
 '78BAE2F8': '1001 S State Hwy 16, Fredericksburg, TX 78624, USA',
 'DE4528F2': '2320 W Main St, Durant, OK 74701, USA',
 'F5539B37': '3702 FM2147, Horseshoe Bay, TX 78657, USA',
 'FE329688': '800 W Interstate 20, Big Spring, TX 79720, USA',
 'C5A903F0': '2503 TX-349, Midland, TX 79706, USA',
 '782881C7': '145 Lehman Rd, Kyle, TX 78640, USA',
 'ECCAB249': '1402 Williams Dr, Georgetown, TX 78628, USA',
 '8E90EF84': '21024 W, 21024 State Hwy 71, Spicewood, TX 78669, USA',
 'B944ED77': '1402 Williams Dr, Georgetown, TX 78628, USA',
 'E7A89C68': '1402 Williams Dr, Georgetown, TX 78628, USA',
 'C41B4C30': '21024 W, 21024 State 

In [37]:
def get_raw_order_to_location_df():
    raw_orders_to_location = order_id_to_pickup_location(orders)
    order_to_location_pairs = list(raw_orders_to_location.items())
    order_to_location_df = pd.DataFrame(order_to_location_pairs, columns=['order_id', 'pickup_location'])
    
    return order_to_location_df
order_to_location_df = get_raw_order_to_location_df()

In [38]:
def splitup_pickup_location():
    order_to_location_df['pickup_location'] = order_to_location_df['pickup_location'].str.replace(',', '')
    
    # Remove "USA" substring
    order_to_location_df['pickup_location'] = order_to_location_df['pickup_location'].str.replace('USA', '')
    
    pattern = r'(?P<address>^.+?)\s(?P<city>\w+\s*\w*)\s(?P<state>[A-Z]{2})\s(?P<zip_code>\d{5})'
    
    df_extracted = order_to_location_df['pickup_location'].str.extract(pattern)
    
    _order_to_location_df = pd.concat([order_to_location_df, df_extracted], axis=1)

    return _order_to_location_df

_order_to_location_df = splitup_pickup_location()
_order_to_location_df

,order_id,pickup_location,address,city,state,zip_code
0,42C2A525,5004 Wesley St Greenville TX 75402,5004 Wesley,St Greenville,TX,75402
1,AA85B890,5004 Wesley St Greenville TX 75402,5004 Wesley,St Greenville,TX,75402
2,773C7330,5004 Wesley St Greenville TX 75402,5004 Wesley,St Greenville,TX,75402
3,EDCB46B1,1001 W Van Alstyne Pkwy Van Alstyne TX 75495,1001 W Van Alstyne Pkwy,Van Alstyne,TX,75495
4,68944F97,800 W Interstate 20 Big Spring TX 79720,800 W Interstate 20,Big Spring,TX,79720
5,ED39CFE2,1301 N U.S. Hwy 281 Marble Falls TX 78654,1301 N U.S. Hwy 281,Marble Falls,TX,78654
6,78BAE2F8,1001 S State Hwy 16 Fredericksburg TX 78624,1001 S State Hwy,16 Fredericksburg,TX,78624
7,DE4528F2,2320 W Main St Durant OK 74701,2320 W Main,St Durant,OK,74701
8,F5539B37,3702 FM2147 Horseshoe Bay TX 78657,3702 FM2147,Horseshoe Bay,TX,78657
9,FE329688,800 W Interstate 20 Big Spring TX 79720,800 W Interstate 20,Big Spring,TX,79720


In [40]:
def add_street_num_col():
    street_num_pattern = r'(?P<street_num>\b\d{3,5}\b)'

    addrs_extracted_df = _order_to_location_df['address'].str.extract(street_num_pattern)

    ADD_order_to_location_df = pd.concat([_order_to_location_df, addrs_extracted_df], axis=1)
    return ADD_order_to_location_df

ADD_order_to_location_df = add_street_num_col()
ADD_order_to_location_df

,order_id,pickup_location,address,city,state,zip_code,street_num
0,42C2A525,5004 Wesley St Greenville TX 75402,5004 Wesley,St Greenville,TX,75402,5004
1,AA85B890,5004 Wesley St Greenville TX 75402,5004 Wesley,St Greenville,TX,75402,5004
2,773C7330,5004 Wesley St Greenville TX 75402,5004 Wesley,St Greenville,TX,75402,5004
3,EDCB46B1,1001 W Van Alstyne Pkwy Van Alstyne TX 75495,1001 W Van Alstyne Pkwy,Van Alstyne,TX,75495,1001
4,68944F97,800 W Interstate 20 Big Spring TX 79720,800 W Interstate 20,Big Spring,TX,79720,800
5,ED39CFE2,1301 N U.S. Hwy 281 Marble Falls TX 78654,1301 N U.S. Hwy 281,Marble Falls,TX,78654,1301
6,78BAE2F8,1001 S State Hwy 16 Fredericksburg TX 78624,1001 S State Hwy,16 Fredericksburg,TX,78624,1001
7,DE4528F2,2320 W Main St Durant OK 74701,2320 W Main,St Durant,OK,74701,2320
8,F5539B37,3702 FM2147 Horseshoe Bay TX 78657,3702 FM2147,Horseshoe Bay,TX,78657,3702
9,FE329688,800 W Interstate 20 Big Spring TX 79720,800 W Interstate 20,Big Spring,TX,79720,800


In [82]:
def get_merged_and_organized_master_df():
    merged_df = pd.merge(master_df, ADD_order_to_location_df, left_on=['Zip', 'street_num'], right_on=['zip_code', 'street_num'], how='inner')

    merged_df.drop(['Site Description', 'City', 'County', 'State', 'address', 'city', 'state', 'Zip', 'zip_code', 'Notes',
     'street_num'], axis=1, inplace=True)

    merged_df = merged_df[['Site #', 'order_id', 'Address', 'pickup_location']]

    return merged_df

merged_df = get_merged_and_organized_master_df()
merged_df

,Site #,order_id,Address,pickup_location
0,16,DE4528F2,2320 West Main,2320 W Main St Durant OK 74701
1,20,42C2A525,5004 Wesley Street,5004 Wesley St Greenville TX 75402
2,20,AA85B890,5004 Wesley Street,5004 Wesley St Greenville TX 75402
3,20,773C7330,5004 Wesley Street,5004 Wesley St Greenville TX 75402
4,32,EDCB46B1,1001 Van Alstyne Parkway,1001 W Van Alstyne Pkwy Van Alstyne TX 75495
5,53,ED39CFE2,1301 Hwy 281 North,1301 N U.S. Hwy 281 Marble Falls TX 78654
6,61,78BAE2F8,1001 Hwy 16 South,1001 S State Hwy 16 Fredericksburg TX 78624
7,62,8E90EF84,21024 Hwy 71 W,21024 W 21024 State Hwy 71 Spicewood TX 78669
8,62,C41B4C30,21024 Hwy 71 W,21024 W 21024 State Hwy 71 Spicewood TX 78669
9,65,ECCAB249,1402 Williams Drive,1402 Williams Dr Georgetown TX 78628


In [78]:
def store_num_to_order_ids(original_dict):
    aggregated_dict = {}
    
    for order_id, value in original_dict.items():
        if value not in aggregated_dict:
            aggregated_dict[value] = set()
        aggregated_dict[value].add(order_id)
    return aggregated_dict


In [79]:
def get_store_num_to_order_ids():
    order_id_to_store_num = merged_df.set_index('order_id')['Site #'].to_dict()
    store_num_to_order_ids_map = store_num_to_order_ids(order_id_to_store_num)
    return store_num_to_order_ids_map
store_num_to_order_ids_map = get_store_num_to_order_ids()
store_num_to_order_ids_map

{'16': {'DE4528F2'},
 '20': {'42C2A525', '773C7330', 'AA85B890'},
 '32': {'EDCB46B1'},
 '53': {'ED39CFE2'},
 '61': {'78BAE2F8'},
 '62': {'8E90EF84', 'C41B4C30'},
 '65': {'B944ED77', 'E7A89C68', 'ECCAB249'},
 '67': {'F5539B37'},
 '68': {'782881C7'},
 '70': {'68944F97', 'FE329688'},
 '71': {'C5A903F0'}}

In [80]:
def add_store_numbers_to_orders(orders, store_num_to_order_ids_map):
    for order in orders:
        order_id = order.get('Order')
        found = False  # Variable to keep track of whether the order_id was found

        for store_num, order_id_set in store_num_to_order_ids_map.items():
            if order_id in order_id_set:
                order['Store Number'] = store_num
                found = True
                break  # Exit the loop once the order_id is found

        if not found:
            order['Store Number'] = 'N/A'
    
    return orders

add_store_numbers_to_orders(orders, store_num_to_order_ids_map)

[{'Order': '42C2A525',
  'Delivered': 'The order was delivered at 11:41 PM on October 1, 2023.',
  'Pick Up Location': '5004 Wesley St, Greenville, TX 75402, USA',
  'Order Details': {'1 × Coca-Cola Can (12 pk-12 oz) (Soda TX)': '$11.29',
   ' 1 × TXB Water Bottle (24 pk) (16.9 oz) (Water TX)': '$7.49',
   ' 1 × Coke Classic Bottle (20oz) (Soda TX)': '$3.19'},
  'Subtotal': '$21.97',
  'Subtotal Tax': '$1.19',
  'Commission (23%)': '-$5.05',
  'Total Customer Refund': '-$0.00',
  'Estimated Payout': '$18.11',
  'Transaction #8113346069 - Delivery': '$18.11',
  'Store Number': '20'},
 {'Order': 'AA85B890',
  'Delivered': 'The order was delivered at 10:08 PM on October 1, 2023.',
  'Pick Up Location': '5004 Wesley St, Greenville, TX 75402, USA',
  'Order Details': {'1 × Moon Pie Double Decker Banana (2.75oz) (Pastry TX)': '$2.29',
   ' 1 × Moon Pie Double Decker Chocolate (2.75oz) (Pastry TX)': '$2.29',
   ' 1 × TXB Sweet Tea Bottle (16oz) (Tea TX)': '$3.29',
   ' 1 × TXB Strawberry Whit